In [15]:
import numpy as np
import random
from random import sample
import math 

In [16]:
def reset1():
    st=np.zeros(16)
    return st


In [17]:
weibull_scale=(2365.08,996.88,713.55,1406.84,343.76,3933.12,828.19,2040.95)
weibull_shape=(414.16,109.25,79.81,115.21,169.81,143.60,43.83,296.48)
tf=(2,6.5,2.5,6,5,3.5,3,3.5)
tp=(0.4,5.42,0.625,0.857,1.25,0.7,0.429,0.875)
time_interval=5
running_time=100000

In [18]:
def step1 (action,st):
    
    for i in range(8):
        f = random.weibullvariate(weibull_scale[i],weibull_shape[i])
        if action[i]==0 :
            if f < st[i]:
                st[i+8]=1
                st[i]=0
            else:
                st[i+8]=0
                st[i] +=5
        if action[i]==1 :
                st[i]=0
            
        
    reward=np.zeros(8)
    for j in range(8):
        if action[j]==1 :
            reward[j]= - (time_interval / tp[j])*tp[j]
        
        if (action[j]==0 and st[j+8]==1):
            reward[j] = - (time_interval / tp[j])*time_interval * math.ceil(tf[j]/time_interval)
            
        else:
            reward[j]=5
    
    return st,reward 
                

In [19]:
def actionll(st):
    a=np.zeros(8)
    T=np.zeros(8)
    for i in range(8):
        T[i]= random.randint(200,4000)
        if st[i] < T[i]:
            a[i]=0
        else:
            a[i]=1
    
    return a , T
        

In [20]:
action_sapce_random = [random.choice([0,1]),random.choice([0,1]),random.choice([0,1]),
                                random.choice([0,1]),random.choice([0,1]),random.choice([0,1]),
                                random.choice([0,1]),random.choice([0,1])]

In [21]:
action_sapce_random

[1, 0, 0, 1, 0, 1, 1, 1]

In [22]:
b=reset1()

In [23]:
a=actionll(b)[1]

a[1]

1674.0

In [24]:
st , r = step1(a,b)

In [25]:
st 

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [26]:
st[0]
st[8]

0.0

In [27]:
state_one_component=[]
state_one_component.append(st[0])
state_one_component.append(st[8])
state_one_component

[0.0, 0.0]

In [28]:
r

array([5., 5., 5., 5., 5., 5., 5., 5.])

In [29]:
timestep = []
timestep.append(a)
timestep.append(st)
episode=[]
episode.append(timestep)
episode

[[array([ 921., 1674., 3261., 3039., 3666., 3661.,  700.,  282.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]]

In [16]:
#def choose_

In [34]:
#monte carlo first visit
def monte_carlo_I ():
    for i in range(8):
        Q = np.matrix(np.full([40000,2],0))
        #Return = np.matrix(np.empty([40000,2]))
        returns = {}
        policy = np.matrix(np.full([40000,2],0.5))
        
        num_episodes=1000 
        
        for epi in range(num_episodes):# Looping through episodes
            # Initializes the state
            s_t = reset1()
            episode = []
            epsilon = 1/(epi+1)
            G = 0 # Store cumulative reward in G (initialized at 0)
            for j in range(running_time//time_interval):
            
                timestep = []
                a_t = action_sapce_random
                s_t , r_t = step1(a_t, s_t)
                state_one_component=[]
                state_one_component.append(s_t[i])
                state_one_component.append(s_t[i+8])
                timestep.append(tuple(state_one_component))
                timestep.append(a_t[i])
                episode.append(timestep)
                 
                state_action = (tuple(state_one_component),a_t[i])
                G += r_t # Increment total reward by reward on current timestep
            
            
                if not state_action in [(x[0], x[1]) for x in episode[0:j]]:
                
                    returns[state_action].append(G)    
                          
                    Q[tuple(state_one_component)][a_t[i]] = sum(returns[state_action]) / len(returns[state_action]) # Average reward across episodes
                
                    Q_list = list(map(lambda x: x[1], Q[tuple(state_one_component)].items())) # Finding the action with maximum value
                    indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]
                    max_Q = random.choice(indices)
                
                    A_star = max_Q # 14.
                
                    for a in policy[state_one_component].items(): # Update action probability for s_t in policy
                        if a[0] == A_star:
                            policy[tuple(state_one_component)][a_t[i]] = 1 - epsilon + (epsilon / abs(sum(policy[tuple(state_one_component)].values())))
                        else:
                            policy[tuple(state_one_component)][a_t[i]] = (epsilon / abs(sum(policy[tuple(state_one_component)].values())))

            
        return policy , Q
            

In [35]:
monte_carlo_I ()

KeyError: ((0.0, 0.0), 1)

In [32]:
#monte carlo first visit
def monte_carlo_I ():
    for i in range(8):
        Q = np.zeros((40000,256) + (2,))
        #Return = np.matrix(np.empty([40000,2]))
        returns = {}
        policy = np.matrix(np.full([40000,2],0.5))
        
        num_episodes=1000 
        
        for epi in range(num_episodes):# Looping through episodes
            # Initializes the state
            s_t = reset1()
            episode = []
            epsilon = 1/(epi+1)
            G = 0 # Store cumulative reward in G (initialized at 0)
            for j in range(running_time//time_interval):
            
                T= []
                timestep = []
                a_t = actionll(s_t)[0]
                T_op = actionll(s_t)[1]
                s_t , r_t = step1(a_t, s_t)
                state_one_component=[]
                state_one_component.append(s_t[i])
                state_one_component.append(s_t[i+8])
                timestep.append(tuple(state_one_component))
                timestep.append(a_t[i])
                episode.append(timestep)
                 
                state_action = (tuple(state_one_component),a_t[i])
                G += r_t # Increment total reward by reward on current timestep
            
            
                if not state_action in [(x[0], x[1]) for x in episode[0:j]]:
                    
                    T.append(T_op[i])
                    returns[state_action].append(G)      
                    Q[tuple(state_one_component)][a_t[i]] = sum(returns[state_action]) / len(returns[state_action]) # Average reward across episodes
                    Q_list = list(map(lambda x: x[1], Q[tuple(state_one_component)].items())) # Finding the action with maximum value
                    indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]
                    max_Q = random.choice(indices)
                
                    A_star = max_Q # 14.
                
                    for a in policy[state_one_component].items(): # Update action probability for s_t in policy
                        if a[0] == A_star:
                            policy[tuple(state_one_component)][a_t[i]] = 1 - epsilon + (epsilon / abs(sum(policy[tuple(state_one_component)].values())))
                        else:
                            policy[tuple(state_one_component)][a_t[i]] = (epsilon / abs(sum(policy[tuple(state_one_component)].values())))
                         
        return policy , Q , T
            

In [33]:
monte_carlo_I ()

KeyError: ((5.0, 0.0), 0.0)